## Import Libraries

In [1]:
from neo4j import GraphDatabase
from pyvis.network import Network
import os
import webbrowser

## Data Loading

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/Blank-Sandbox.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")
database = config.get("NEO4J_DATABASE", "neo4j")

## Customer Relationship Database Class

In [5]:
class customerRelationsDatabase:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def initialise_graph(self):
        with self.driver.session() as session:
            session.run(
                """
                    MATCH (n)
                    DETACH DELETE n

                    CREATE 
                        (sophie:Customer {name: 'Sophie'}),
                        (jeremy:Customer {name: 'Jeremy'}),
                        (clem:Customer {name: 'Clem'}),
                        (antonio:Customer {name: 'Antonio'}),
                        (rosa:Customer {name: 'Rosa'}),
                        (karl:Customer {name: 'Karl'}),
                        (nye:Customer {name: 'Nye'})

                    CREATE 
                        (shureE215:Product {category: 'Audio|Wired|Earphones|Products', mfr: 'Shure', model: 'E215'}),
                        (boseQC3511:Product {category: 'Audio|Wired and Wireless|Earphones|Products', mfr: 'Bose', model: 'QC3511'}),
                        (appleEarpods:Product {category: 'Audio|Wireless|Earphones|Products', mfr: 'Apple', model: 'Earpods'})
                    
                    CREATE 
                        (sophie)-[:BOUGHT]->(shureE215),
                        (jeremy)-[:BOUGHT]->(shureE215),
                        (clem)-[:BOUGHT]->(boseQC3511),
                        (antonio)-[:BOUGHT]->(shureE215),
                        (rosa)-[:BOUGHT]->(shureE215),
                        (karl)-[:BOUGHT]->(shureE215),
                        (karl)-[:BOUGHT]->(appleEarpods),
                        (nye)-[:BOUGHT]->(appleEarpods)
                """
            )

    def visualise_graph(self):
        with self.driver.session() as session:
            query = """
                MATCH (c:Customer)-[:BOUGHT]->(p:Product)
                RETURN c.name AS customer, p.model AS model, p.mfr AS mfr, p.category AS category
            """
            results = session.run(query)

            net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black', directed=True)
            net.force_atlas_2based()

            added_nodes = set()
            added_edges = set()  # Track (source, target)

            for record in results:
                cust = record["customer"]
                model = record["model"]
                mfr = record["mfr"]
                category = record["category"]
                product_label = f"{mfr} {model}"

                # Add nodes
                if cust not in added_nodes:
                    net.add_node(cust, label=cust, shape='ellipse', color='lightblue')
                    added_nodes.add(cust)

                if product_label not in added_nodes:
                    net.add_node(product_label, label=f"{product_label}\n{category}", shape='box', color='orange')
                    added_nodes.add(product_label)

                # Add edge only once
                edge_key = (cust, product_label)
                if edge_key not in added_edges:
                    net.add_edge(cust, product_label, label='BOUGHT')
                    added_edges.add(edge_key)

        # Save and open
        path = os.path.abspath("../../Playground/output/customer_product_graph.html")
        net.write_html(path)
        webbrowser.open("file://" + os.path.abspath(path))

    def close(self):
        self.driver.close()

In [6]:
db = customerRelationsDatabase(uri, username, password)
db.initialise_graph()
db.visualise_graph()
db.close()